In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero
dir_ = '../../../data'
group_dir_ = os.path.join(dir_, 'groups/random')

In [2]:
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'
df = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

num_user = len(svd['uid'].unique())
num_user

220

In [12]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10', '20']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [4]:
pop_dict = {}
for i in df['tid'].unique():
    pop_dict[i] = len(df[df['tid']==i])/num_user

In [5]:
svd['count'] = 1
svd[:5]

,uid,tid,rating,count
0,0,3,2.544071,1
1,0,7,2.545822,1
2,0,18,2.152490,1
3,0,20,1.882396,1
4,0,31,2.410806,1


In [6]:
# Count POP to DF
tid_list = []
pop_list = []
for i in df['tid'].unique():
    tid_list.append(i)
    pop_list.append(len(df[df['tid']==i])/num_user*5)
    
d = {'tid': tid_list, 'rating': pop_list}
df_pop = pd.DataFrame(data=d)
df_pop = df_pop.sort_values(by=['rating'], ascending=False)
df_pop[:10]

,tid,rating
166,166,3.840909
7209,7209,3.295455
350,350,3.113636
3761,3761,3.045455
80,80,3.022727
1017,1017,2.840909
1753,1753,2.750000
428,428,2.727273
200,200,2.681818
3159,3159,2.659091


In [7]:
top_percent = 0.01
long_tail = df_pop[int(len(df_pop)*top_percent):]
long_tail = long_tail['tid'].unique()
len(long_tail), len(df_pop), len(df_pop)-len(long_tail)

(54420, 54969, 549)

In [8]:
lambdas = []
for i in range(11):
    lambdas.append(i/10)
lambdas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [ ]:
for lambda_ in tqdm(lambdas):
    top_n_size = 4
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
    threshold = 0.0
    avg_pops = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        pops = []
        for j in (range(len(groups))):
            group = groups[j]
            top_n_list = top_n_lists[j]
            for k in range(len(group)):
                for l in range(top_n_size):
                    tid = top_n_list[l]
                    pops.append(pop_dict[tid])                    
        avg_pop = sum(pops)/len(pops)
        avg_pops.append(avg_pop)
        
    print(avg_pops)

In [13]:
HitNOVs = []
for lambda_ in tqdm(lambdas):
    top_n_size = 50
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
    HitNOV = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        
        nov = 0
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0:
                        nov += (1-pop_dict[tid])
        
        HitNOV.append(nov/top_n_size/(len(groups)*len(groups[0])))
    HitNOVs.append(HitNOV)
    
for HitNOV in HitNOVs:
    print(HitNOV)


[0.04923636363636365, 0.04923636363636367, 0.049236363636363654, 0.049236363636363564, 0.04942887205387194, 0.04895517385839958, 0.04952819865319849, 0.04851893939393937, 0.049105371900826304, 0.049105371900826325]
[0.05797348484848481, 0.054887878787878736, 0.053807575757575755, 0.052101515151515045, 0.052406565656565526, 0.051335986594050954, 0.05254082491582478, 0.05106607744107736, 0.05122975206611552, 0.0503355371900825]
[0.07326666666666659, 0.06280151515151504, 0.05954772727272721, 0.05708333333333318, 0.05644360269360255, 0.055616673648931574, 0.055584595959595855, 0.054454545454545394, 0.054086776859503985, 0.0513702479338841]
[0.08558106060606048, 0.07447651515151504, 0.0707954545454544, 0.06478560606060593, 0.06310606060606067, 0.06112442396313359, 0.06023863636363626, 0.05909680134680139, 0.058105371900826375, 0.05336859504132212]
[0.09833106060606038, 0.08334090909090892, 0.07949469696969692, 0.07481060606060598, 0.07100589225589227, 0.06672308336824471, 0.066200757575757

In [11]:
diversities = []
novelties = []
for lambda_ in tqdm(lambdas):
    top_n_size = 4
    top_n_items_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_items = pd.DataFrame() 

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]   
            if len(top_n_items) == 0:
                top_n_items = rating_table
            else:
                top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()

        top_n_items_g.append(top_n_items)   
    
    diversity = []
    for i in range(len(groups_n)):
        top_n_items = top_n_items_g[i]
        diversity.append(len(top_n_items)/len(df['tid'].unique()))
    diversities.append(diversity)

    novelty = []
    for i in range(len(groups_n)):
        top_n_items = top_n_items_g[i]
        novelty.append(len(top_n_items[top_n_items['tid'].isin(long_tail)])/len(df['tid'].unique()))
    novelties.append(novelty)
    
for diversity in diversities:
    print(diversity)
print()
for novelty in novelties:
    print(novelty)


[7.276828758027252e-05, 7.276828758027252e-05, 7.276828758027252e-05, 7.276828758027252e-05, 7.276828758027252e-05, 7.276828758027252e-05, 7.276828758027252e-05, 7.276828758027252e-05, 7.276828758027252e-05, 7.276828758027252e-05]
[0.00010915243137040878, 0.00010915243137040878, 0.00010915243137040878, 9.096035947534064e-05, 9.096035947534064e-05, 9.096035947534064e-05, 9.096035947534064e-05, 9.096035947534064e-05, 9.096035947534064e-05, 9.096035947534064e-05]
[0.00027288107842602196, 0.0002001127908457494, 0.00014553657516054504, 0.0001273445032654769, 0.00010915243137040878, 0.00010915243137040878, 0.00010915243137040878, 0.00010915243137040878, 0.00010915243137040878, 9.096035947534064e-05]
[0.00045480179737670325, 0.0004002255816914988, 0.00032745729411122633, 0.00027288107842602196, 0.00032745729411122633, 0.00027288107842602196, 0.00021830486274081756, 0.0001819207189506813, 0.00021830486274081756, 0.00010915243137040878]
[0.0006731066601175208, 0.0005821463006421802, 0.00052757

In [11]:
tid_pd = pd.DataFrame(data={'tid': df['tid'].unique()})
tid_pd = tid_pd.sort_values(by=['tid']).reset_index()
long_tail_list = tid_pd['tid'].isin(long_tail)
long_tail_list = list(long_tail_list)
long_tail_list.count(True)

54420

In [12]:
diversities = []
novelties = []
for lambda_ in tqdm(lambdas):
    top_n_size = 50
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
    diversity = []
    novelty = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        
        hits = np.zeros(len(df['tid'].unique()), dtype=bool)
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0:
                        hits[int(tid)] = True
        
        diversity.append(list(hits).count(True)/len(df['tid'].unique()))
        novelty.append(list(hits&long_tail_list).count(True)/len(df['tid'].unique()))
    diversities.append(diversity)
    novelties.append(novelty)
    
for diversity in diversities:
    print(diversity)
print()
for novelty in novelties:
    print(novelty)


[0.0009096035947534065, 0.0009096035947534065, 0.0009096035947534065, 0.0009096035947534065, 0.0009096035947534065, 0.0009096035947534065, 0.0009096035947534065, 0.0009096035947534065, 0.0009096035947534065, 0.0009096035947534065]
[0.001400789535920246, 0.001273445032654769, 0.0012188688169695646, 0.0011461005293892921, 0.0012370608888646328, 0.0011824846731794284, 0.0011642926012843603, 0.0011824846731794284, 0.0011461005293892921, 0.0010005639542287471]
[0.002110280339827903, 0.0017828230457166768, 0.0017282468300314723, 0.001546326111080791, 0.0016736706143462678, 0.001546326111080791, 0.0014917498953955866, 0.0014917498953955866, 0.0013644053921301096, 0.0012006767450744966]
[0.002983499790791173, 0.002728810784260219, 0.002401353490148993, 0.0023649693463588567, 0.0022922010587785842, 0.0020738961960377667, 0.002110280339827903, 0.0018737834051920173, 0.0018919754770870855, 0.0014735578235005185]
[0.004038639960705125, 0.0038385271698593753, 0.0032927650130073313, 0.0033109570849